# Opis notatnika

Jest to docelowy notatnik, służący przeprowadzaniu eksperymentów rozwiązania. Planowane są następujące porówania:
- SA vs HC
- SA vs RS
- Wpływ wykorzystywanych pokemonów na wyniki
- Wpływ pokemonów legendarnych na wyniki
- Wpływ ilość zmienianych pokemonów w drużynie po przegranej na wyniki
- Ewaluacja nalepszych druzyń (znalezionych przez nasz algorytm)

### Importy

In [1]:
import sys
from pathlib import Path
import pandas as pd

SRC = (Path("..")).resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


In [2]:
from classes.pokemon_team import PokemonTeam
from visualization.utils import summarize
from data.data import get_pokemons
from report.reports import PdfReport
from experiments.sa_experiments import run_multiple_runs, compare_to_hill_climb, compare_to_random_search
from visualization.plots import visualize_opponents_typing_distribution, visualize_opponents_stat_sums_violin
from constants import REPORT_DIR

In [3]:
from solvers import SimulatedAnnealingPokemonSolver
from solvers import HillClimbingPokemonSolver
from solvers import RandomSearchPokemonSolver

### Parametry startowe

In [4]:
pokemons = get_pokemons()
all_pokemons = get_pokemons(include_only_final_evolutions=False)
pokemons_and_legendaries = get_pokemons(include_legendary=True, include_only_final_evolutions=True)

In [5]:
print("Ilość dostępnych pokemonów bazowo:", len(pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje):", len(all_pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne):", len(pokemons_and_legendaries))

Ilość dostępnych pokemonów bazowo: 359
Ilość dostępnych pokemonów (wszystkie ewolucje): 731
Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne): 427


#### Wielkość przestrzeni przeszukiwań

- Lista oponentów będzie składała się z 50 losowych drużyn, w celu przeprowadzenia szybszych eksperymentów. 
- Następnie najlepsze z drużyn zostaną wziętę pod uwagę w porównaniu i zostaną przetestowane na większym zbiorze, w celu weryfikacji ich skuteczności.
- Każda solver, będzie działał na takim samym budżecie równym 300 iteracją (w celu szybkiego szukania drużyn)
- Z wstępnych eksperymentów wyszło, że duża temperatura negatywnie wpływała na solver SA, więc zaczynać będziemy z wartością 0.3 i zmniejszać będziemy ją o 0.80

In [6]:
OPPONENTS_LIMIT = 50
BUDGET = 300
INITIAL_TEMPERATURE = 0.3
COOLING = 0.8

In [7]:
sa_solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=INITIAL_TEMPERATURE,
    min_temperature=1e-3,
    alpha=COOLING,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=0,
    )

hc_solver = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=0)

rs_solver = RandomSearchPokemonSolver(trials=BUDGET, opponents_limit=OPPONENTS_LIMIT, unique_types=True)

### Opponents (50 drużyn)

In [8]:
pokemons = get_pokemons()
opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

# Porównanie SA vs HC (10 runów)

In [9]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa_vs_hc = compare_to_hill_climb(pokemons, sa_solver, hc_solver, opponents=opponents, runs=10)

sum_sa_hc = summarize(df_sa_vs_hc)

report = PdfReport(str(REPORT_DIR / "sa_vs_hc_last_evolutions.pdf"), title="SA vs HC (same opponents)")

report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {INITIAL_TEMPERATURE}, cooling = {COOLING}
- restarts = 0
- patience = 50
- same opponents for all experiments
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa_vs_hc, "SA vs HC (same start per run): per-run results")
report.add_dataframe(sum_sa_hc, "SA vs HC: summary stats")


report.write()
print(f"Saved: {REPORT_DIR / 'report_sa_experiments.pdf'}")

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Comparing SA solver to Hill Climbing solver...
 Run 1/10...
  SA fitness: 0.2415944540727903, HC fitness: 0.49635087719298243
  SA team: ['Bellossom', 'Porygon-Z', 'Alomomola', 'Lucario', 'Donphan', 'Hydreigon']
  HC team: ['Blissey', 'Emboar', 'Stunfisk', 'Vaporeon', 'Drifblim', 'Aromatisse']
 Run 2/10...
  SA fitness: 0.2625649350649351, HC fitness: 0.46631578947368413
  SA team: ['Wobbuffet', 'Clawitzer', 'Mamoswine', 'Eelektross', 'Pidgeot', 'Kricketune']
  HC team: ['Metagross', 'Abomasnow', 'Slaking', 'Heracross', 'Mudsdale', 'Alomomola']
 Run 3/10...
  SA fitness: 0.22547445255474455, HC fitness: 0.5534953703703703
  SA team: ['Hariyama', 'Tyranitar', 'Dragalge', 'Comfey', 'Audino', 'Trevenant']
  HC team: ['Mudsdale', 'Gigalith', 'Abomasnow', 'Hariyama', 'Blissey', 'Wobbuffet']
 Run 4/10...
  SA fitness: 0.2174814814814815, HC fitness: 0.5040772014475272
  SA team: ['Lickilicky', 'Golurk', 'Aromatisse', 'Blaziken', 'Archeops', 'Hypno']
  HC team: ['Tyranitar', 'Blissey', 'Stunf

# Porównanie SA vs RS

In [10]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa_vs_rs = compare_to_random_search(pokemons, sa_solver, rs_solver, opponents=opponents, runs=10)

sum_sa_rs = summarize(df_sa_vs_rs)

report = PdfReport(str(REPORT_DIR / "sa_vs_rs_last_evolutions.pdf"), title="SA vs RS (same opponents)")
report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {INITIAL_TEMPERATURE}, cooling = {COOLING}
- restarts = 0
- patience = 50
- same opponents for all experiments
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa_vs_rs, "SA vs RS (same start per run): per-run results")
report.add_dataframe(sum_sa_rs, "SA vs RS: summary stats")


report.write()

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Comparing SA solver to Random Search solver...
 Run 1/10...
  SA fitness: 0.24778378378378377, RS fitness: 0.3916196136701337
  SA team: ['Avalugg', 'Rhyperior', 'Granbull', 'Floatzel', 'Maractus', 'Bouffalant']
  RS team: ['Snorlax', 'Rhyperior', 'Sunflora', 'Durant', 'Oricorio', 'Wobbuffet']
 Run 2/10...
  SA fitness: 0.3596501457725948, RS fitness: 0.3995862068965517
  SA team: ['Swampert', 'Gogoat', 'Gumshoos', 'Throh', 'Wobbuffet', 'Glaceon']
  RS team: ['Blissey', 'Krookodile', 'Conkeldurr', 'Forretress', 'Walrein', 'Heatmor']
 Run 3/10...
  SA fitness: 0.1826843100189036, RS fitness: 0.32974276527331187
  SA team: ['Umbreon', 'Sunflora', 'Castform', 'Flygon', 'Beedrill', 'Hariyama']
  RS team: ['Pinsir', 'Dhelmise', 'Greninja', 'Mudsdale', 'Turtonator', 'Blissey']
 Run 4/10...
  SA fitness: 0.22892921960072596, RS fitness: 0.34444444444444444
  SA team: ['Yanmega', 'Lucario', 'Umbreon', 'Dunsparce', 'Mudsdale', 'Tangrowth']
  RS team: ['Torterra', 'Passimian', 'Turtonator', 'Wea

# Uwzględnienie pokemonów legendarnych (tylko SA)

In [9]:
OPPONENTS_LIMIT = 100

opponents = PokemonTeam.generate_unique_teams(
    pokemons_and_legendaries,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

sa_solver_legendaries = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.1,
    min_temperature=1e-3,
    alpha=0.5,
    iters_per_temp=10,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=3,
    )

In [10]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa = run_multiple_runs(pokemons_and_legendaries, sa_solver_legendaries, opponents=opponents, runs=10)

sum_sa = summarize(df_sa)

report = PdfReport(str(REPORT_DIR / "sa_with_legendaries.pdf"), title="SA including legendary pokemons")

report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {0.1}, cooling = {0.5}
- restarts = 3
- patience = 50
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa, "SA with legendaries")
report.add_dataframe(sum_sa, "SA: summary stats")


report.write()

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 10 runs of the SA solver...
 Run 1/10...
  Best fitness: 0.3791472868217054
  Team: ['Lunala', 'Blastoise', 'Groudon', 'Braviary', 'Nihilego', 'Throh']
 Run 2/10...
  Best fitness: 0.42151866151866146
  Team: ['Blissey', 'Gyarados', 'Zekrom', 'Nidoking', 'Throh', 'Xerneas']
 Run 3/10...
  Best fitness: 0.37330645161290327
  Team: ['Tyranitar', 'Mewtwo', 'Zekrom', 'Whiscash', 'Tangrowth', 'Exploud']
 Run 4/10...
  Best fitness: 0.4248113207547169
  Team: ['Wailord', 'Tangrowth', 'Aurorus', 'Mewtwo', 'Guzzlord', 'Xerneas']
 Run 5/10...
  Best fitness: 0.42966625463535224
  Team: ['Blissey', 'Necrozma', 'Blastoise', 'Kyurem', 'Stunfisk', 'Hariyama']
 Run 6/10...
  Best fitness: 0.45223577235772355
  Team: ['Buzzwole', 'Nihilego', 'Mewtwo', 'Glalie', 'Zygarde', 'Arceus']
 Run 7/10...
  Best fitness: 0.40495778045838354
  Team: ['Crabominable', 'Gogoat', 'Wobbuffet', 'Snorlax', 'Nihilego', 'Drifblim']
 Run 8/10...
  Best fitness: 0.4124199743918054
  Team: ['Blissey', 'Aromatisse', 

# Wpływ ilości zmienianych pokemonów po przegranej (tylko SA)

In [ ]:
OPPONENTS_LIMIT = 50
BUDGET = 150
INITIAL_TEMPERATURE = 0.3
COOLING = 0.8
ITERS_PER_TEMP = 4

In [7]:
pokemons = get_pokemons()
opponents_normal = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

opponents_legendaries = PokemonTeam.generate_unique_teams(
    pokemons_and_legendaries,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

In [8]:
GRID_SEARCH_PARAMS = {
    'neighbor_replacements': [1, 2, 3],
    'initial_temperature': [0.1, 0.3, 0.5],
    'cooling': [0.5, 0.8, 0.9],
    'legendaries': [False, True],
}

In [ ]:
from sklearn.model_selection import ParameterGrid

for params in ParameterGrid(GRID_SEARCH_PARAMS):
    cooling = params['cooling']
    initial_temperature = params['initial_temperature']
    legendaries = params['legendaries']
    neighbour_replacements = params['neighbor_replacements']

    if legendaries:
        opponents = opponents_legendaries
    else:
        opponents = opponents_normal

    opponents_typings = visualize_opponents_typing_distribution(opponents)
    opponents_stats = visualize_opponents_stat_sums_violin(opponents)

    sa_solver_different_replacements = SimulatedAnnealingPokemonSolver(
        initial_temperature=initial_temperature,
        min_temperature=1e-3,
        alpha=cooling,
        iters_per_temp=ITERS_PER_TEMP,
        max_evaluations=BUDGET,
        neighbor_replacements=neighbour_replacements,
        opponents_limit=OPPONENTS_LIMIT,
        unique_types=True,
        patience=50,
        restarts=0,
        )


    df_sa = run_multiple_runs(pokemons_and_legendaries, sa_solver_legendaries, opponents=opponents, runs=10)

    sum_sa = summarize(df_sa)

    report = PdfReport(str(REPORT_DIR / "sa_with_legendaries.pdf"), title="SA including legendary pokemons")

    report.add_text(
        f"""Setup
    - opponents_limit = {len(opponents)}
    - runs = 10
    - budget = {BUDGET}
    - SA: initial_temperature = {0.1}, cooling = {0.5}
    - restarts = 3
    - patience = 50
    """
    )

    report.add_figure(opponents_typings)
    report.add_figure(opponents_stats)


    report.add_dataframe(df_sa, "SA with legendaries")
    report.add_dataframe(sum_sa, "SA: summary stats")


    report.write()

0.5 0.1 False 1
